In [6]:
import pandas as pd
df = pd.read_csv('airtable_papers.csv')
len(df)

1249

In [8]:
df = df[df['URL'].notna()]
len(df)

1024

In [4]:
df['URL'].values[:10]

array(['https://royalsocietypublishing.org/doi/full/10.1098/rsif.2012.0132',
       'https://jeb.biologists.org/content/223/20/jeb226654',
       'https://www.nature.com/articles/s42004-019-0202-8  https://science.sciencemag.org/content/315/5810/348',
       'https://zslpublications.onlinelibrary.wiley.com/doi/abs/10.1111/j.1469-7998.1973.tb04553.x',
       'https://www.yumpu.com/en/document/read/40748212/toward-biomimetic-architecture-annapujadascat',
       'https://www.uregina.ca/science/biology/people/faculty-research/brigham-mark/files/Woods_and_Brigham_2004_LITC.pdf',
       'https://www.tandfonline.com/doi/pdf/10.1080/15376490590928598',
       'https://www.tandfonline.com/doi/full/10.4161/psb.4.9.9426?scroll=top&needAccess=true',
       'https://www.tandfonline.com/doi/abs/10.1271/bbb.100692',
       'https://www.tandfonline.com/doi/abs/10.1080/17088180809434786'],
      dtype=object)

In [11]:
for index, row in df.iterrows():
    urls = row['URL'].split(' ')
    if len(urls) >= 2:
        df = df.drop(index=index)
        for url in urls:
            data = [row['DOI'], row['Paper title'], row['Abstract'], row['Journal'], url,
                   row['Functions'], row['PDF'], row['Press release']]
            df.append(data)

In [18]:
for url in df['URL'].values:
    if len(url.split(' ')) > 2:
        print(u)

In [20]:
df = df.reset_index()

In [21]:
df.to_csv('cleaned_airtable.csv')